<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/ba/May2008gasolineCA.jpg/379px-May2008gasolineCA.jpg" width="200px;" alt="gas prices"/>

#   `lesson01`:  Collecting, organizing, and analyzing data

## Objectives

### Objectives

1. Identify the pieces of a Pandas dataframe for a set of data.
2. Interpret data through plotting. 
3. Apply data filtering techniques to prepare the data for analysis.
4. Organize multiple data sets for analysis.
5. Construct a comparison between two sets of data.

### Example Questions

1. What are the column types in your dataframe?
2. How do you plot a column of data?
3. Which data needs to be modified in your dataframe?
4. How do you plot two time series?
5. How would you correlate two series of data?

## Highlevel topics

- Data importing and storage
- Data cleaning
- Data plotting
- Plot manipulation
- Data analysis using built-in tools

## Synopsis

You are a data scientist working for a DC think tank, and your team is studying technology and energy policy.  To prepare for an upcoming energy sumit you are studying the relationship between **US fuel prices** and **fuel efficiency**, measured in miles-per-gallon.

#### Your Task

Your goal is to identify trends in two different datasets on **US fuel prices** and **fuel efficiency**.

## Datasets

In this session two datasets will be used:
- Automotive Trends Report
    - This dataset provides **miles per gallon** on light-duty vehicles
    - https://www.epa.gov/automotive-trends/explore-automotive-trends-data
    - https://www.epa.gov/automotive-trends/about-automotive-trends-data
- Retail motor gasoline and on-highway diesel fuel prices
    - This dataset provides **fuel prices**
    - https://www.eia.gov/totalenergy/data/browser/index.php?tbl=T09.04#/
    - (section 9.4) https://www.eia.gov/totalenergy/data/monthly/index.php

Both of these datasets are already downloaded for you as `table_export.csv` (for MPG data) and `MER_T09_04.csv` (for fuel price data).  However, they could easily be downloaded from the above sites.  For example, the following shell command would download the second dataset:
```
wget https://www.eia.gov/totalenergy/data/browser/csv.php\?tbl\=T09.04 -O MER_T09_04.csv
```

## Getting Started


### Setting up Python

First, import a few Python packages that we'll use throught the course.  By convention these are abbreviated on import.

- `matplotlib` and the interface `matplotlib.pyplot` for plotting
- `numpy` for numerical functions and arrays
- `pandas` for data structures and analysis
- `seaborn` for additional plotting and improved figures

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

%matplotlib inline

### Import data

Here we will import the data with Pandas `read_csv` function and store as a *dataframe*.  

What is a *dataframe*?  It's a storage container (provided by Pandas) that functions like a table.  It can also be viewed as a dictionary.  Pandas dataframes have lots of useful functions, many of which we won't use in this lesson (see [Pandas dataframe documenation](http://pandas.pydata.org/pandas-docs/stable/reference/frame.html) for more details).

In [3]:
ecodf = pd.read_csv('table_export.csv')

### Example dataframe

Let's construct a mock dataframe to highlight some basic functionality.

In [4]:
mydf = pd.DataFrame(
    {'month': ['January', 'February', 'March'],
     'temperature': [20, 30, 40],
     'snowfall': [12.5, 15, 'trace']
    }
)

We can inspect the dataframe in a few different ways:

- `mydf.info()` shows a highlevel view of the dataframe as a data structure
- `mydf` or `print(mydf)` will give a tabular view

In [5]:
mydf

In [6]:
pd.to_datetime('2017', format='%Y')

In [7]:
mydf.info()

In [8]:
mydf

We can access a given column of a dataframe using the bracket notation with the column label.

In [9]:
mydf['temperature']

Also notice that each column is a Pandas *series*.  A series is simply array of values with an index to those values.

In [10]:
type(mydf['temperature'])

#### Pandas methods

In the following we'll be doing mainly three things to data stored like `mydf`:

1. formatting the data
2. setting an index
3. cleaning the data

We'll work with the example dataframe for now.  Later, we'll work with the datasets described above and we'll also merge data and introduce some analytics.

In [11]:
mydf.info()

#### (1)
Let's *format* the data so that the `month` is an actual datetime format.  We can do this using [`pd.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html).
For this we need to refer to the string format of dates in Python's `time` format:
https://docs.python.org/3/library/time.html#time.strftime

Notice that `%B` means the month name.

In [12]:
pd.to_datetime('2019 January', format='%Y %B')

In [13]:
pd.to_datetime(mydf['month'], format='%B')

Notice, the above command doesn't  actually change the column of our dataframe `mydf`.

In [14]:
mydf['month']

To add a year, we would use `%Y`.  To change our dataframe, we set the column equal to the new series.

In [15]:
mydf['month'] = pd.to_datetime(mydf['month']+'2019', format='%B%Y')

In [16]:
mydf

In [17]:
mydf.info()

#### (2)

Each column of a Pandas dataframe is a series and the default is to index this series with integer indices starting at 0.  We can see what the current index values are by accessing the dataframe's `index` attribute (not a function).  We can also set the index to another set of labels, say the months using the dataframe's `set_index()` function.

In [18]:
mydf.index

In [19]:
mydf.set_index('month', inplace=True)

Notice we used `inplace=True` above so it modified `mydf` instead of making a new object.  We can look at the modified index and dataframe:

In [20]:
mydf.index

In [21]:
mydf

#### (3)

Notice that the last value of snowfall is "trace" (a small amount of snow, but no measurable accumulation).  Unfortunately, this isn't very helpful -- we cannot take the average (or many of the other summary statistics) of a string.

In [22]:
mydf['snowfall'].mean()

Since "trace" means a small amount, it's fairly reasonable to represent it as 0.  So we're going to construct a function that we can `apply()` to each entry.  Let's check to see if the entry is "trace" and if so, set it to 0.0.

In [23]:
def f(x):
    if x == 'trace':
        return 0.0
    else:
        return x

mydf['snowfall'] = mydf['snowfall'].apply(f)
mydf

Now that "trace" is removed, we can take the average.

In [24]:
mydf['snowfall'].mean()

## The fuel economy dataset

Using the practice from the `mydf` example, let's take a look at the `ecodf` dataframe we obtained above from importing the fuel economy dataset.

In [25]:
ecodf.info()

In [26]:
ecodf

Take a look at the columns --- we'll be considering the 'Adjusted MPG' for our analysis.

In [27]:
ecodf.columns

### Plot the MPG

Let's try to plot the values of `Adjusted MPG` using the `plot()` method for series.

In [28]:
ecodf['Adjusted MPG'].plot()

#### How can we improve this?

1. It looks like we're indexing this by integers (the x-axis).  A more helpful view would be years (or dates).
2. From the dataset above, all vehicle types are being plotted (so there are multiple values corresponding to each year).  Try plotting only for the vehicle type `Car SUV`, for example.
3. The plot needs **labels** (axes, legend) and improved formatting (look, size, font).

#### (1) formatting the dates

Let's format the `Model Year` column and set it as our index.

In [29]:
pd.to_datetime(ecodf['Model Year'], format='%Y')

Since the most recent data is marked as preliminary, it's a string that isn't being recognized as a year.
We'll have to work around that manually.

In [30]:
'Prelim. 2017'.split()[-1]

In [31]:
def f(t):
    if 'Prelim.' in t:
        t = t.split(' ')[-1]
    timestamp = pd.datetime.strptime(t, '%Y')
    return timestamp

ecodf['Model Year'] = ecodf['Model Year'].apply(f)

In [32]:
ecodf.set_index('Model Year', inplace=True)
ecodf

In [33]:
ecodf['Adjusted MPG'].plot()

#### (2)

We still have multiple vehicle types being plotted for each year (the large oscillating pattern).
Now check to see where the `Vehicle Type` is equal to `Car SUV` and only plot that data.

In [34]:
ecodf[
ecodf['Vehicle Type']=='Car SUV'
]['Adjusted MPG'].plot()

#### (3)

Note that changing the index automatically applied the index column label as the x-axis label.

But, there's still a lot we can do to improve the plot with more labels and other
visual formatting changes.

First, we'll adjust the image size, add axis labels/legend, and make the line thicker.


In [35]:
fig = plt.figure(figsize=(10,10))
ax = fig.gca()

ecodf[
    ecodf['Vehicle Type']=='Car SUV'
]['Adjusted MPG'].plot(ax=ax, linewidth=4)

ax.legend()
plt.ylabel('Miles Per Gallon')

We can also change the fontsize and the general look.

In [36]:
sns.set(font_scale=2)

fig = plt.figure(figsize=(10,10))
ax = fig.gca()

ecodf[
    ecodf['Vehicle Type']=='Car SUV'
]['Adjusted MPG'].plot(ax=ax, linewidth=4)

ax.legend()
plt.ylabel('Miles per Gallon')
plt.xlabel('Year')

The data has a lot of small variation that can make it
harder to see the overall trend.  Let's plot smoothed
data from a rolling average 
by combining the Pandas series functions `.rolling()` and `.mean()`.

In [37]:
sns.set(font_scale=2)

fig = plt.figure(figsize=(10,10))
ax = fig.gca()

ecodf[
    ecodf['Vehicle Type']=='Car SUV'
]['Adjusted MPG'].rolling(10).mean().plot(ax=ax, linewidth=4)

ax.legend()
plt.ylabel('Miles per Gallon')
plt.xlabel('Year')

## Your turn, the fuel prices dataset

The goal of this portion of the notebook is to construct a correlation between **fuel prices** and **fuel efficiency**.  We've already imported and formatted the fuel efficiency dataset, but you'll be starting from the original .csv for the fuel prices dataset.

To do this consider the following challenge questions:

1. How do you format the fuel price data with a `datetime` index?  It may be helpful to distinguish between monthly values and yearly averages (the yearly averages end in "13" for this dataset).

2. How should you handle missing data in the `Value` column?

3. Do you see a trend in regular unleaded gas prices? (`MSN` column is `RUUCUUS` for regular unleaded gas)

4. Find a correlation between the **fuel price** and **fuel efficiency**.  To do this you may want to combine the relevant values from the different dataframes using `pd.merge_asof()` and then use the function `.corr()` on the combined dataframe.

5. Try to plot the **fuel price** and **fuel efficiency** on the same plot, but with different y-axis scales -- do you observe a correlation?

6. Plot **fuel price** and **fuel efficiency** using a rolling average, for example `rolling(5).mean()` on a Pandas series to display a 5 year rolling average.  See above for an example of rolling average.  Plot the rolling averages like you plotted the values in the previous question.

5. Use seaborn's `jointplot()` to plot MPG vs Price to deduce a correlation.

### Getting started

First import the data

In [38]:
pricedf = pd.read_csv('MER_T09_04.csv')

Next, do two things:

1. Make a column called `Data Type` and mark it as `AVG` if the year string contains a `13`.
2. For each row that's an `AVG`, format the year string in one way.

In [43]:
def f(x):
    x = str(x)
    if x[-2:] == '13':
        return 'AVG'
    else:
        return ''
pricedf['Data Type'] = pricedf['YYYYMM'].apply(f)

def f(x):
    y = str(x['YYYYMM'])
    if x['Data Type'] == 'AVG':
        return pd.to_datetime(y[:4], format='%Y')
    else:
        return pd.to_datetime(y, format='%Y%m')

pricedf['Date'] = pricedf.apply(f, axis=1)

Now check to see what all of the `AVG` `Value` numbers look like.

In [45]:
pricedf[
    pricedf['Data Type'] == 'AVG'
]['Value']

For the next step you'll want to 

1. try to convert a number to a float
2. if the convertion doesn't work, then use not-a-number (np.nan)

In [49]:
try:
    y = float('1.23')
except:
    print('this should not print')
print(f'y={y}')

try:
    y = float('trace')
except:
    y = np.nan
    print('converted to nan')
    
print(f'y={y}')